## MongoDB資料庫連線

In [1]:
import re
from pymongo import MongoClient 

#預設就是自己
client = MongoClient('10.120.28.12',27017)
database = client['test']
collection =database['news']

#for post in collection.find():
    #print post['title']

## 實戰:給關鍵字進行搜尋+分群

In [97]:
tag1 = "柯文哲"
tag2 = "雙城"

#查詢滿足多重條件的語法是{"$and":[{條件1},{條件2},{條件3},...]}
#另有$or, $not, $nor
#$all在pymongo已廢止, 因為與$and功能重複

for post in collection.find(
    {"$and":[                   
            {"content":{"$regex":tag1}},
            {"content":{"$regex":tag2}},
            {"date":{"$regex":"2016"}}
            ]},{"_id":0}):
    print post['title']

In [ ]:
#搜尋+計算字詞頻率
import re
import jieba
from sklearn.feature_extraction.text import CountVectorizer
jieba.set_dictionary('E:/dics/dict.txt.big.txt')  #切換至中文繁體字庫
jieba.load_userdict("E:/dics/dict_keyw.txt")       #加入自建詞庫

#給關鍵字----------------------------------------
tag1 = "蔡英文"
tag2 = "選舉"
#-----------------------------------------------

title=[]
content =[]
#把資料庫東西抓出來
for post in collection.find(
    {"$and":[                   
            {"content":{"$regex":tag1}},
            {"content":{"$regex":tag2}},
            #{"date":{"$regex":"2016"}},
            ]},{"_id":0}): 
    summary = post['content']
    content.append('/'.join(jieba.cut(summary)))
    title.append(post['title'])
print type(title)
print title[0]
print title[1]
print "---------------------------------"

# 準備計算字詞頻率
# get_feature_names() 可以看"特徵詞有哪些" (『一個字』及『標點符號』會自動被去除掉)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(content) #裡面要放list
#for i in vectorizer.get_feature_names():
    #print i, #印出特徵詞
print "----------------------"    

#特徵值總共有多少個
print "Feature_num",len(vectorizer.get_feature_names())

#文章數
print "文章數:",len(title)

#透過toarray()可看到詞頻矩陣的结果
print X.toarray()

Building prefix dict from E:\dics\dict.txt.big.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u56ec63d90e89b3354142c30b3eb89a90.cache
Loading model cost 0.583 seconds.
Prefix dict has been built succesfully.


In [15]:
#計算TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(X) # 塞入TF-IDF 函數進行計算 #对应的tfidf矩阵

#把權重矩陣存起來
weight = tfidf.toarray()

#整合"特徵值"，"count"，"weight"
a=0
#整合第某篇新聞的"特徵值"，"count"，"weight"
for i in vectorizer.get_feature_names():
    if int(X.toarray()[0][a]) > int(0):  #出現次數 > 0 再印出來
        print i ," ",X.toarray()[0][a]," Weight: ",weight[0][a]
    a=a+1

fu   1  Weight:  0.0558221413872
一下   1  Weight:  0.0390432061459
一定   1  Weight:  0.0267822297732


KeyboardInterrupt: 

In [22]:
#clustering-1
from sklearn import cluster
c = cluster.KMeans(n_clusters=6)
k_data = c.fit_predict(weight)  # weight 直接拿來用 或放 X 有多算一層TF_IDF 分群效果會好一點

In [29]:
#clustering-2
for idx, grp in enumerate(k_data):
    if grp == 6:
        print title[idx]

## 單純把資料抓出來進行TF-IDF計算

In [27]:
#計算字詞頻率
import re
import jieba
from sklearn.feature_extraction.text import CountVectorizer
jieba.set_dictionary('E:/dics/dict.txt.big.txt')  #切換至中文繁體字庫
jieba.load_userdict("E:/dics/dict_keyw.txt")       #加入自建詞庫

title=[]
content =[]
#把資料庫東西抓出來
for post in collection.find():
    summary = post['title']
    title.append('/'.join(jieba.cut(summary)))
print type(title)
print title[0]
print title[1]
print title[2]
print "---------------------------------"

# 準備計算字詞頻率
# get_feature_names() 可以看"特徵詞有哪些" (『一個字』及『標點符號』會自動被去除掉)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(title) #裡面要放list
for i in vectorizer.get_feature_names():
    print i, #印出特徵詞
print "----------------------"    

#透過toarray()可看到詞頻矩陣的结果
print X.toarray()

Building prefix dict from E:\dics\dict.txt.big.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u56ec63d90e89b3354142c30b3eb89a90.cache
Loading model cost 0.600 seconds.
Prefix dict has been built succesfully.


<type 'list'>
2013/,/台灣/起舞
燦爛/跨年/,/歡動/全台
《/蘋果/》/民調/：/過去/1/年/，/最讓/你/感到/不滿/的/是/？
---------------------------------
00 000 007 01 02 03 039 05 06 07 08 09 099 0元 10 100 1000 100m 101 102 106 108 109 1094 10c 11 110 111 112 113 115 116 1162 118 119 12 120 121 122 123 1231 125 128 1286 128gb 12c 12年國教 13 130 1300 131 1314 136 139 13k 14 140 1400 143 1431 145 146 1471 148 15 150 1500 157 16 160 1600 161 163 164 1658 167 1688 17 18 180 1800 184 186 1878 188 1881 1885 19 1920 1968 198 199 1996 1k 20 200 2012 20121231 2013 20130 201301 20130102 20130103 20130104 20130105 20130107 20130108 20130109 20130110 20130111 20130112 20130114 20130115 20130116 20130118 20130119 20130121 20130122 20130123 20130124 20130125 20130126 20130128 20130129 20130130 2015 2016 2021 206 2060 21 210 2125 214 215 218 21k 22 222 2244 2250 2265 22k 23 2300 233 234 24 245 249 25 250 2500 252 255 25k 26 2600 27 270 2700 277 279 28 280 2827 286 2895 29 290 293 298 2pm 2q 30 300 3000 3002 300i 300ml 304 305 30k 31 318 319 

In [31]:
#計算TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
X = vectorizer.fit_transform(title)  # X 是算好的字詞頻率矩陣
tfidf = transformer.fit_transform(X) # 塞入TF-IDF 函數進行計算 #对应的tfidf矩阵

#可以看算出來TF-IDF 權重矩陣
print tfidf.toarray()
#把權重矩陣存起來
weight = tfidf.toarray()
print type(weight)
print "---------------------------------------------"
#可以透過以下方法進行取值
print weight[0][1]
print "---------------------------------------------"

#特徵值總共有多少個
print len(vectorizer.get_feature_names())

#整合"特徵值"，"count"，"weight"
a=0
#for i in vectorizer.get_feature_names():
    #print "特徵值：",i ," 在第幾篇新聞出現: ",X.toarray()[0][a],X.toarray()[1][a],X.toarray()[2][a]," 權重值: ",weight[0][a],"  ",weight[1][a],"  ",weight[2][a]
    #a=a+1

import time    
#整合第某篇新聞的"特徵值"，"count"，"weight"
for i in vectorizer.get_feature_names():
    if int(X.toarray()[0][a]) > int(0):  #出現次數 > 0 再印出來
        print "特徵值：",i ," 出現次數: ",X.toarray()[0][a]," 權重值: ",weight[0][a]
    a=a+1
        #time.sleep(0.5)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
<type 'numpy.ndarray'>
---------------------------------------------
0.0
---------------------------------------------
19784
特徵值： 2013  出現次數:  1  權重值:  0.491775426193


KeyboardInterrupt: 

## 抓出TF-IDF高的詞存入原本的資料庫中→{Key:'TF-IDF',Value:'柯文哲/腳踏車.....'}

In [55]:
print type(X)
print type(weight)
print "----------------------------------"
print X.toarray()
print "----------可以抓出第幾篇文章------------------------"
print weight[0]

help(list)

print type(vectorizer.get_feature_names())
#print vectorizer.get_feature_names()

<class 'scipy.sparse.csr.csr_matrix'>
<type 'numpy.ndarray'>
----------------------------------
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
----------可以抓出第幾篇文章------------------------
[ 0.  0.  0. ...,  0.  0.  0.]
Help on class list in module __builtin__:

class list(object)
 |  list() -> new empty list
 |  list(iterable) -> new list initialized from iterable's items
 |  
 |  Methods defined here:
 |  
 |  __add__(...)
 |      x.__add__(y) <==> x+y
 |  
 |  __contains__(...)
 |      x.__contains__(y) <==> y in x
 |  
 |  __delitem__(...)
 |      x.__delitem__(y) <==> del x[y]
 |  
 |  __delslice__(...)
 |      x.__delslice__(i, j) <==> del x[i:j]
 |      
 |      Use of negative indices is not supported.
 |  
 |  __eq__(...)
 |      x.__eq__(y) <==> x==y
 |  
 |  __ge__(...)
 |      x.__ge__(y) <==> x>=y
 |  
 |  __getattribute__(...)
 |      x.__getattribute__('name') <==> x.name
 |  
 |  __getitem__(

In [73]:
a = [2, 4, 3, 5, 1]
print sorted(a)

#a.sort()
print a
print "----------------------"
b=[[2, 4, 3, 5, 1],
   [3, 4, 3, 10, 1],
   [2, 4, 7, 5, 1],
   [2, 8, 3, 6, 1],  
]

print sorted(b)



[1, 2, 3, 4, 5]
[2, 4, 3, 5, 1]
----------------------
[[2, 4, 3, 5, 1], [2, 4, 7, 5, 1], [2, 8, 3, 6, 1], [3, 4, 3, 10, 1]]


## Clustering

In [243]:
from sklearn import cluster
c = cluster.KMeans(n_clusters=15)
k_data = c.fit_predict(weight)  # weight 直接拿來用 或放 X 有多算一層TF_IDF 分群效果會好一點

In [259]:
for idx, grp in enumerate(k_data):
    if grp == 6:
        print title[idx]

燦爛/跨年/,/歡動/全台
《/蘋果/》/民調/：/過去/1/年/，/最/讓/你/感到/不滿/的/是/？
揮別/痛楚/,/我/相信/我能/飛
回歸/職場/,/華隆/人/不/放棄
拒當/英雄/,/「/只求/順利/畢業/」
喪子傷/慟/,/「/期待/司法/正義/」
勇敢/媽/：/前面/有/無限/希望/「/早睡/躲/飢餓/」/,/如願/快樂/出遊
甩/32/公斤/,/「/要/hold/住/身材/」
登山/失溫/,/68/歲/男/生死未卜
迎接/2013/,/玉山/連/3/波/瑞雪/又/一個/冷氣團/,/後天/報到
國道/北上/,/估/下午/湧車潮
帶/超量/菸酒/入境/,/今起/除罪
反/媒體/壟斷/,/上千/學子/守夜/忍受/低溫/「/再/冷/也/要/表達/心聲/」
司馬/觀點/：/中國/孤獨/崛起/,/（/江春/男/）
辦美/免簽/,/上/錯/網站/枉花/千元/代辦/網/一堆/也/叫/ESTA/,/「/./gov/」/才/是/官網
蘋論/：/ROC/是/啥
蘋論/：/政府/怕/啥
露營車/當/禮車/,/蜜月/趴/趴/走/「/不怕/訂/不到/房/」/,/要/在/墾丁/迎/曙光
《/蘋果/數/》/,/「/中國/為/世界/強國/」/,/8/成/大陸/人/不/認同
北斗/導航/,/提供/亞太區/服務
我/很/不爽
商圈/明星臉/,/小/李準/基/最/像/鳳眼/好帥/,/飲料/至少/多賣/3/成
共軍/對台/導彈/失準/,/怪罪/美/GPS1996/台海/危機/揭祕/,/中國/改/自/製/北斗/衛星
空/拋/棉被/,/搶/買
全球/首創/芝心餅/皮/PIZZA/，/好/口碑/全/台/熱銷/大破/千萬/份/！
捷運/廁性/侵/,/狼辯/她/沒/反抗/同學/臉書/重逢/,/騙/出遊/,/害/懷孕
「/想鬧/一下/」/煙火/嚇友/誤燒/6/車
騎樓/嘿咻遭/逮/,/女/誣性/侵/呻吟/未/反抗/,/見警/還落/跑/,/被/揭/賣淫
抽/水毀/鄰宅/,/瓏/山林/判賠/186/萬
低溫/9/℃/,/凍死/摔車/西施
除塵/紙/,/買/1/送/1
20121231/開彩/結果/與/獎金/分配/表
男/走上/國道/,/遭/3/車/輾斃/拖/百米
士官/酒駕/撞/8/車/,/軍方/要警/封口
母悲求/國家/幫管/賊/兒/國一生/累犯/,/嗆/「/偷東西/沒什麼/」
動物/姓氏/,/100/元

## Practice_WordCount & TF-IDF
## 重要觀念:
- 字詞頻率一定要先算出來，才能計算TF-IDF
- TF_IDF是把字詞頻率『標準化』的概念，一篇文章中，某字詞雖然只出現一次，但是卻可能很具代表性，TF-IDF會讓他分數變很高
- vectorizer.get_feature_names()會把『一個字』及『標點符號』自動去除掉

## 套建概念:
- scikit-learn包进行TF-IDF分词权重计算主要用到了两个类：CountVectorizer和TfidfTransformer。其中
- CountVectorizer是通过fit_transform函数将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在第i个文本下的词频。即各个词语出现的次数，通过get_feature_names()可看到所有文本的关键字，通过toarray()可看到词频矩阵的结果。简例如下：

In [15]:
#計算字詞頻率
from sklearn.feature_extraction.text import CountVectorizer
import jieba
jieba.set_dictionary('E:/dics/dict.txt.big.txt')  #切換至中文繁體字庫
jieba.load_userdict("E:/dics/dict_keyw.txt")       #加入自建詞庫
corpus = ['上櫃IC設計類股新兵報到',
          '陳彥廷上櫃成民國95年，，，john',
          '另外,九齊',
]

y=[]
# 把元素抓出來進行切詞
for i in corpus:
    y.append(' '.join(jieba.cut(i)))

# y list裡面存放切好的字詞['上櫃 ic 設計 類股 新兵 報告','陳彥廷 上櫃 成 上櫃','']
print type(y)
print y[0]
print y[1]
print y[2]
print "---------------------------------"

# 準備計算字詞頻率
# get_feature_names() 可以看"特徵詞有哪些" (『一個字』及『標點符號』會自動被去除掉)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(y) #裡面要放list
for i in vectorizer.get_feature_names():
    print i, #印出特徵詞
print "----------------------"    

#透過toarray()可看到詞頻矩陣的结果
print X.toarray()


Building prefix dict from E:\dics\dict.txt.big.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.u56ec63d90e89b3354142c30b3eb89a90.cache
Loading model cost 0.640 seconds.
Prefix dict has been built succesfully.


<type 'list'>
上櫃 IC設計 類股 新兵 報到
陳彥廷 上櫃 成 民國 95 年 ， ， ， john
另外 , 九齊
---------------------------------
95 ic設計 john 上櫃 九齊 另外 報到 新兵 民國 陳彥廷 類股 ----------------------
[[0 1 0 1 0 0 1 1 0 0 1]
 [1 0 1 1 0 0 0 0 1 1 0]
 [0 0 0 0 1 1 0 0 0 0 0]]


In [16]:
#計算TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
X = vectorizer.fit_transform(y)  # X 是算好的字詞頻率矩陣
tfidf = transformer.fit_transform(X) # 塞入TF-IDF 函數進行計算

#可以看算出來TF-IDF 權重矩陣
print tfidf.toarray()
#把權重矩陣存起來
weight = tfidf.toarray()
print "---------------------------------------------"
#可以透過以下方法進行取值
print weight[0][1]
print "---------------------------------------------"

#整合"特徵值"，"count"，"weight"
a=0
for i in vectorizer.get_feature_names():
    print i ,"  ",X.toarray()[0][a],X.toarray()[1][a],X.toarray()[2][a],"  ",weight[0][a],"  ",weight[1][a],"  ",weight[2][a]
    a=a+1

[[ 0.          0.46735098  0.          0.35543247  0.          0.
   0.46735098  0.46735098  0.          0.          0.46735098]
 [ 0.46735098  0.          0.46735098  0.35543247  0.          0.          0.
   0.          0.46735098  0.46735098  0.        ]
 [ 0.          0.          0.          0.          0.70710678  0.70710678
   0.          0.          0.          0.          0.        ]]
---------------------------------------------
0.467350981811
---------------------------------------------
95    0 1 0    0.0    0.467350981811    0.0
ic設計    1 0 0    0.467350981811    0.0    0.0
john    0 1 0    0.0    0.467350981811    0.0
上櫃    1 1 0    0.35543246785    0.35543246785    0.0
九齊    0 0 1    0.0    0.0    0.707106781187
另外    0 0 1    0.0    0.0    0.707106781187
報到    1 0 0    0.467350981811    0.0    0.0
新兵    1 0 0    0.467350981811    0.0    0.0
民國    0 1 0    0.0    0.467350981811    0.0
陳彥廷    0 1 0    0.0    0.467350981811    0.0
類股    1 0 0    0.467350981811    0.0    0.0

In [24]:
print type(X)
print type(weight)
X.toarray()
print "-----------------------------"
tfidf.toarray()
print "-----------------------------"
print type(vectorizer.get_feature_names())

<class 'scipy.sparse.csr.csr_matrix'>
<type 'numpy.ndarray'>
-----------------------------
-----------------------------
<type 'list'>
